# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [17]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [2]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head(10)

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,0,Vila Velha,BR,73.94,-20.3297,-40.2925,Hotel Prainha
1,1,El Llano,MX,74.43,20.0500,-99.3000,Hotel Reco Inn
2,2,Atlantic Beach,US,73.62,30.3344,-81.3987,One Ocean Resort & Spa
3,4,Hermanus,ZA,71.80,-34.4187,19.2345,Aloe guest house
4,5,Rikitea,PF,74.26,-23.1203,-134.9692,Chez Bianca & Benoit
5,13,Cape Town,ZA,75.61,-33.9258,18.4232,Townhouse Hotel
6,18,Ca Mau,VN,72.72,9.1769,105.1500,Khách sạn Song Ngọc
7,19,Vaini,TO,80.76,-21.2000,-175.2000,Keleti International Resort
8,38,Tomatlan,MX,78.31,19.9333,-105.2500,Hotel Hacienda Vieja Tomatlan
9,40,Labuan,MY,77.36,5.2767,115.2417,Labuan Avenue Hotel


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [29]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.8,
    color = "City",
    hover_cols = ["Hotel Name","Country", "Current Description"])

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [31]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[(vacation_df["City"] == "Kununurra")]
vacation_end = vacation_df.loc[(vacation_df["City"] == "Mount Isa")]
vacation_stop1 = vacation_df.loc[(vacation_df["City"] == "Mareeba")]
vacation_stop2 = vacation_df.loc[(vacation_df["City"] == "Mackay")]
vacation_stop3 = vacation_df.loc[(vacation_df["City"] =="Victoria Point")]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [32]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat(
    [
        vacation_start,
        vacation_stop1,
        vacation_stop2,
        vacation_stop3,
        vacation_end,
    ],
    ignore_index = True
)

# Display sample data
itinerary_df.head(5)

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,163,Kununurra,AU,84.18,-15.7667,128.7333,Kununurra Country Club Resort
1,67,Mareeba,AU,84.49,-17.0000,145.4333,The Gateway Hotel
2,378,Mackay,AU,86.68,-21.1500,149.2000,Mackay Oceanside Central Apartment Hotel
3,528,Victoria Point,AU,76.24,-27.5833,153.3000,Seminara Appartments
4,287,Mount Isa,AU,83.97,-20.7333,139.5000,Ibis Styles


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [33]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["City","Lat", "Lng"]].copy()

# Display sample data
waypoints_df

,City,Lat,Lng
0,Kununurra,-15.7667,128.7333
1,Mareeba,-17.0000,145.4333
2,Mackay,-21.1500,149.2000
3,Victoria Point,-27.5833,153.3000
4,Mount Isa,-20.7333,139.5000


## Use GeoViews to create map that shows the four cities in the itinerary

In [157]:
# Configure the map plot by using the itineraty_df
waypoints_map = waypoints_df.hvplot.points(
        "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 370,
    size = 150,
    scale = 1,
    color = "City",)

In [158]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [36]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [37]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '559827599b1a4261bc6b24fd88514b26',
 'waypoints': '-15.7667,128.7333|-17.0,145.4333|-21.15,149.2|-27.5833,153.3|-20.7333,139.5'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [38]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params).json()

# Convert the API response to JSON format
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [128.7333, -15.7667], 'original_index': 0},
     {'location': [145.4333, -17], 'original_index': 1},
     {'location': [149.2, -21.15], 'original_index': 2},
     {'location': [153.3, -27.5833], 'original_index': 3},
     {'location': [139.5, -20.7333], 'original_index': 4}],
    'units': 'metric',
    'distance': 6571614,
    'distance_units': 'meters',
    'time': 228548.498,
    'legs': [{'distance': 2933696,
      'time': 94525.054,
      'steps': [{'from_index': 0,
        'to_index': 7,
        'distance': 156,
        'time': 11.265,
        'instruction': {'text': 'Drive southwest on Button Drive.'}},
       {'from_index': 7,
        'to_index': 33,
        'distance': 1292,
        'time': 99.602,
        'instruction': {'text': 'Turn left onto Ironwood Drive.'}},
       {'from_index': 33,
        'to_index': 49,
        'distance': 1125,
        'time': 72.651,
        'instruc

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [39]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]
legs

[[[128.733768, -15.766984],
  [128.733649, -15.767166],
  [128.733589, -15.767254],
  [128.733573, -15.767371],
  [128.733728, -15.767565],
  [128.734016, -15.767864],
  [128.734163, -15.768011],
  [128.734194, -15.768122],
  [128.734267, -15.768013],
  [128.734528, -15.767801],
  [128.735258, -15.767307],
  [128.735783, -15.766975],
  [128.736019, -15.766867],
  [128.736109, -15.766819],
  [128.736388, -15.766752],
  [128.736618, -15.766753],
  [128.736959, -15.766791],
  [128.737431, -15.766903],
  [128.738559, -15.767235],
  [128.738756, -15.767299],
  [128.738811, -15.767317],
  [128.739565, -15.767587],
  [128.740253, -15.76786],
  [128.740955, -15.768119],
  [128.741362, -15.768239],
  [128.741718, -15.768282],
  [128.742157, -15.768252],
  [128.742595, -15.768156],
  [128.74302, -15.768058],
  [128.74326, -15.768023],
  [128.743461, -15.768042],
  [128.7446, -15.768276],
  [128.745388, -15.768438],
  [128.745488, -15.768459],
  [128.745125, -15.77065],
  [128.744599, -15.774347]

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [47]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs:
    for coordinate in leg:
        longitude.append(coordinate[0])
        latitude.append(coordinate[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [49]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["Longitude"] = longitude
route_df["Latitude"] = latitude
# Display sample data
route_df.head()

,Longitude,Latitude
0,128.733768,-15.766984
1,128.733649,-15.767166
2,128.733589,-15.767254
3,128.733573,-15.767371
4,128.733728,-15.767565


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [153]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(
    route_df,
).opts(
    width = 1200,
    height = 650,
    line_width = 3,
    color = "red"
)

In [154]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map * route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)
   .Path.I   :Path   [Longitude,Latitude]